In [2]:
import pandas as pd # data frames
import numpy as np

In [3]:
# read data
df = pd.read_csv("https://raw.githubusercontent.com/askoshiyama/mli-cohort3/master/boston.csv")
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,T1
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Repeated K Fold CV (5 outer, 5 inner, repeat 5 times):
RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [4]:
# defining vars
input_vars = ["V1", "V2", "V3", "V4", "V5", "V6", "V7",
              "V8", "V9", "V10", "V11", "V12", "V13"]
target = ["T1"]

In [5]:
X = df.drop(labels=target, axis=1)
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [6]:
y = df.drop(labels=input_vars, axis=1)
y.head()

,T1
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [30]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from interpret.glassbox import ExplainableBoostingRegressor 
from xgboost import XGBRegressor
from sklearn.svm import SVR

# define the models for selection along with their hyperparameters
# we try to find meaningful ranges for each hyperparameter. Among the models chosen for selection,
# we also include an MLP neural network as it presents the most recent advancement in supervised learning domain.
models_and_parameters = {
    'dummy': (DummyRegressor(), {}),
    'linReg': (LinearRegression(), {}),
    'ridge': (Ridge(), 
              {'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'svr': (SVR(kernel='rbf'),
              {'C': [0.01, 0.05, 0.1, 0.5, 1], 
               'gamma': [0.001, 0.0001, 0.00001]}),
    'krr': (KernelRidge(), 
              {'kernel': ["poly","rbf"],
               'degree': [2,3,4],
               'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'dtr': (DecisionTreeRegressor(),
              {"min_samples_split": [2, 10, 20, 40], 
               "max_depth": [2, 6, 8],
               "min_samples_leaf": [1, 20, 40, 100],
               "max_leaf_nodes": [None, 5, 20, 100]}),
    'rf': (RandomForestRegressor(n_estimators=100),
              {'max_depth': [5, 10, 50, 100, 200, 500]}),
    'gbr': (GradientBoostingRegressor(),
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'ebr': (ExplainableBoostingRegressor(), {}),
    'xgbr': (XGBRegressor(), 
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'mlp': (MLPRegressor(early_stopping=True),
                {'hidden_layer_sizes': [(64,),(128,)],
                 'activation': ['relu'],
                 'solver':['lbfgs', 'adam'], 
                 'alpha':[0.0001],
                 'batch_size':['auto'], 
                 'learning_rate':['adaptive', 'constant'],
                 'learning_rate_init':[0.1, 0.01, 0.001], 
                 'max_iter':[1600]})
}

In [27]:
from sklearn import metrics

# performance metrics
perf_metrics = {#"MAE": metrics.mean_absolute_error, 
                #"EVS": metrics.explained_variance_score, 
                "MSE": metrics.mean_squared_error,
                #"MDAE": metrics.median_absolute_error, 
                "R2": metrics.r2_score
               }
metric_greater_is_better = \
               {#"MAE": False, 
                # "EVS": True, 
                "MSE": False,
                # "MDAE": False, 
                "R2": True
               }
scorer_metrics = {}
for pf in perf_metrics:
    scorer_metrics[pf] = metrics.make_scorer(score_func=perf_metrics[pf], greater_is_better=metric_greater_is_better[pf])

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, GridSearchCV
import numpy as np
from sklearn import metrics
from time import time


#######################################
# configuration
REPEAT = 1
SEED = None  # will ensure different results on each iteration
K_FOLD_NUM = 5
REFIT_METRIC = 'MSE'
#######################################

# use nested cross-validation for grid search and validation
# creates 5 folds for estimating generalization error
outer_folds = RepeatedKFold(n_repeats=REPEAT, n_splits=K_FOLD_NUM, random_state=SEED)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_folds = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

i, z, innercv_results, inner_best_model = 0, 0, {}, {}
df_outer_results = pd.DataFrame(index=[0], columns=["Model", "Time"] + list(perf_metrics.keys()))

X_arr, y_arr = X.values, y.values

# outer loop
for (out_train, out_test) in outer_folds.split(X_arr, y_arr):
    print(f"Repetition {int(i / K_FOLD_NUM) + 1} Fold {i % K_FOLD_NUM + 1}")
    i += 1  # iteration
    # separation: train, test
    Xtrain, Xtest = X_arr[out_train], X_arr[out_test]
    ytrain, y_test = y_arr[out_train], y_arr[out_test]
    
    # inner loop -- all models
    fold_id = "fold_" + str(i)
    innercv_results[fold_id] = {}
    inner_best_model[fold_id] = {}
    start, end = [], []
    
    for model_name, (model, params) in models_and_parameters.items():
        start.append(time())
        print("\t", model_name)
        innercv_results[fold_id][model_name] = GridSearchCV(estimator=model, 
                                                            param_grid=params, 
                                                            scoring=scorer_metrics, 
                                                            iid=False,
                                                            cv=inner_folds, 
                                                            refit=REFIT_METRIC)    
        innercv_results[fold_id][model_name].fit(Xtrain, ytrain.ravel())
        inner_best_model[fold_id][model_name] = innercv_results[fold_id][model_name].best_estimator_
        end.append(time())        
    
    # outer loop prediction    
    for v, (model_name, (model, _)) in enumerate(models_and_parameters.items()):
        # prediction
        y_test_pred = inner_best_model[fold_id][model_name].predict(Xtest)
        df_outer_results.loc[z, "Model"] = model_name
        df_outer_results.loc[z, "Time"] = end[v] - start[v]
        
        # compute performance metrics
        for pf in perf_metrics.keys():
            df_outer_results.loc[z, pf] = perf_metrics[pf](y_test, y_test_pred)
        z += 1        

# final organisation
df_outer_results[list(perf_metrics.keys()) + ["Time"]] = df_outer_results[list(perf_metrics.keys()) + ["Time"]].astype(float)
print("done.")

Repetition 1 Fold 1
	 mlp


C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't co

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't co

Repetition 1 Fold 2
	 mlp


C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't co

...

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't co

Repetition 1 Fold 5
	 mlp


C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't co

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1600) reached and the optimization hasn't co

done.


In [86]:
df_outer_results.pivot_table(index=["Model"], aggfunc="mean")

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
dtr,7.358596e-01,2.906646,1.918784,22.241523,0.734435,3.436193
dummy,-8.881784e-17,6.656448,4.762839,84.628499,-0.007098,0.012196
ebr,8.260339e-01,2.503000,1.706848,14.565834,0.822692,45.543682
gbr,8.830504e-01,2.137486,1.525193,9.952860,0.881654,14.825601
krr,8.164701e-01,2.610738,1.815452,15.292612,0.814656,41.772621
linReg,7.268719e-01,3.385594,2.596291,23.206449,0.722423,0.018948
rf,8.754108e-01,2.255113,1.590400,10.621005,0.873531,10.822156
ridge,7.267465e-01,3.382341,2.567266,23.234433,0.722080,0.835336
svr,3.156644e-01,4.919969,2.960138,60.699866,0.277519,1.197190


In [12]:
df_outer_results

,Model,Time,MAE,EVS,MSE,MDAE,R2
0,mlp,249.467521,3.260511,0.703334,23.263250,2.462454,0.701680
1,mlp,227.036417,2.596037,0.788284,15.701160,1.848053,0.786165
2,mlp,211.663650,2.859156,0.854957,13.541250,2.254715,0.854891
3,mlp,218.519864,2.897830,0.730711,24.610911,1.833671,0.730238
4,mlp,262.683425,2.925630,0.814116,15.354390,2.379098,0.814085


In [29]:
df_outer_results

,Model,Time,MSE,R2
0,mlp,353.114889,23.763467,0.772439
1,mlp,381.211406,72.664328,0.012156
2,mlp,409.817692,24.650344,0.651899
3,mlp,350.170197,19.764604,0.770144
4,mlp,311.624131,25.860588,0.694229


In [32]:
df_outer_results

,Model,Time,MSE,R2
0,mlp,2239.103773,15.904127,0.840225
1,mlp,2189.910840,19.508750,0.782384
2,mlp,2316.255329,23.309288,0.740941
3,mlp,2245.900880,23.097715,0.634955
4,mlp,2666.705462,8.047284,0.898293


In [22]:
innercv_results['fold_4']['mlp'].best_estimator_

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(128,), learning_rate='adaptive',
             learning_rate_init=0.001, max_iter=1200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [40]:
innercv_results['fold_1']['mlp'].best_estimator_

MLPRegressor(activation='relu', alpha=0.0005, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(64,), learning_rate='adaptive',
             learning_rate_init=0.1, max_iter=1600, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [105]:
best_models_mean = df_outer_results.pivot_table(index=["Model"], aggfunc="mean")
best_model_refit_metric = best_models_mean[REFIT_METRIC].idxmin()
best_models_mean[best_models_mean.index == best_model_refit_metric]

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
xgbr,0.889541,2.133136,1.538067,9.526195,0.886969,13.208702


In [109]:
df_outer_results[df_outer_results['Model'] == best_model_refit_metric]

,Model,Time,MAE,EVS,MSE,MDAE,R2
9,xgbr,8.915425,2.062250,0.885993,10.370350,1.246335,0.884310
19,xgbr,20.354385,2.276010,0.839982,13.130137,1.503645,0.839037
29,xgbr,14.576690,2.000264,0.922849,6.435288,1.716067,0.922535
39,xgbr,11.498063,1.945446,0.905258,6.977491,1.469378,0.905249
49,xgbr,10.698944,2.381712,0.893625,10.717710,1.754912,0.883713


In [117]:
# TODO at work
innercv_results['fold_3']['xgbr']

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthr...
                         'n_estimators': [50, 100, 150]},
             pre_dispatch='2*n_jobs', refit='MSE', return_train_score=False,
             scoring={'EVS': make_scorer(explained_variance_score),
                      'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
                      'MDAE': make_scorer(median_absolute_error, greater_is_better=False),
    

TODO: pick best model according to MSE and print best model params
XGBoost vs GBT:
https://stats.stackexchange.com/questions/282459/xgboost-vs-python-sklearn-gradient-boosted-trees